<center>
    <h1> ILI285 - Computación Científica I  / INF285 - Computación Científica </h1>
    <h2> Linear Systems of Equations </h2>
    <h2> [[S]cientific [C]omputing [T]eam](#acknowledgements)</h2>
    <h2> Version: 1.1</h2>
</center>

## Table of Contents
* [Introduction](#intro)
* [Direct Methods](#DM)
* [LU](#lu)
* [Palu](#palu)
* [Cholesky](#cholesky)
* [Iterative Methods](#im)
* [Convergence Analysis](#ca)
* [Exercises](#ex)
* [Acknowledgements](#acknowledgements)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<div id='intro' />
## Introduction

In our last IPython Notebook we learned how to solve 1D equations. Now, we'll go to the next level and will learn how to solve not just <i>one</i> equation, but a <i>system</i> of linear equations. This is a set of $n$ equations involving $n$ variables wherein all the equations must be satisfied at the same time. You probably know how to solve small 2D systems with methods such as substitution and reduction, but in practical real-life situations it's very likely that you'll find problems of bigger dimensions. As usual, we'll present some useful methods for solving systems of linear equations below.

<div id='DM' />
## Direct Methods

Firstly, we will study _direct methods_. They compute the analytic solution of the system (from here comes the name **direct**) limited only by the loss of numerical precision, because of the arithmetic operations performed by the computer. Their counterpart is the _iterative methods_, which calculate an approximate solution that evolves iteratively converging to the real solution.

<div id='lu' />
### LU decomposition

Given the matrix $A \in \mathbb{R}^{n \times n}$ square and non singular, the main goal of this method involves finding a decomposition like $A = L U$ where $L,U \in  \mathbb{R}^{n \times n}$ are lower and upper triangular matrices respectively.

The algorithm to perform this decomposition is basically a modified version of _Gaussian Elimination_. It basically iterates through the first $n-1$ columns, making $0$ all the entries below the main diagonal. This is accomplished by performing row operations. 

In [ ]:
def lu_decomp(A, show=False):
    N,_ = A.shape
    U = np.copy(A)
    L = np.identity(N)
    if show:
        print('Initial matrices')
        print('L = '); print(np.array_str(L, precision=2, suppress_small=True))
        print('U = '); print(np.array_str(U, precision=2, suppress_small=True))
        print('----------------------------------------')
    #iterating through columns
    for j in range(N-1):
        #iterating through rows
        for i in range(j+1,N):
            L[i,j] = U[i,j]/U[j,j]
            U[i] -= L[i,j]*U[j] 
            if show:
                print('L = '); print(np.array_str(L, precision=2, suppress_small=True))
                print('U = '); print(np.array_str(U, precision=2, suppress_small=True))
                print('----------------------------------------')
    return L,U

Once the decomposition is done, solving a linear system like $A x = b$ is straightforward:

$$A x = b \rightarrow L U x = b \ \ \text{ if we set } \ \  U x = c \rightarrow L c = b \ \ \text{ (solve for c) } \ \rightarrow U x = c$$

and as you might know, solving lower and upper triangular systems can be easily performed by back-substitution and forward-subsitution respectively. 

In [ ]:
"""
Solves a linear system A x = b, where A is a
triangular (upper or lower) matrix
"""
def solve_triangular(A, b, upper=True):
    n = b.shape[0]
    x = np.zeros_like(b)
    if upper==True:
        #perform back-substitution
        x[-1] = (1./A[-1,-1]) * b[-1]
        for i in range(n-2, -1, -1):
            x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,i+1:] * x[i+1:]))
    else:
        #perform forward-substitution
        x[0] = (1./A[0,0]) * b[0]
        for i in range(1,n):
            x[i] = (1./A[i,i]) * (b[i] - np.sum(A[i,:i] * x[:i]))
    return x

def solve_lu(A, b, show=False):
    L,U = lu_decomp(A, show)
    # L.c = b with c = U.x
    c = solve_triangular(L, b, upper=False)
    x = solve_triangular(U, c)
    return x

Let's now try our implementations. We begin by creating a random 100$\times$100 linear system:

In [ ]:
A = np.random.random((3,3))
b = np.ones(3)

and then we compute the solution with our LU solver, and aditionally with the NumPy solver which computes the solution using LAPACK routines.

In [ ]:
lu_sol = solve_lu(A,b, show=True)
np_sol = np.linalg.solve(A,b)

in order to compare these huge vectors, we use the Euclidean metric as follows:

In [ ]:
np.linalg.norm(lu_sol - np_sol)

which is a very good result!

This method has two important facts to be noted:

1. Computing the LU decomposition requires $2n^3/3$ floating point operations. Can you check that?
2. When computing the LU decomposition you can see the instruction `L[i,j] = U[i,j]/U[j,j]`. Here we divide an entry below the main diagonal by the _pivot_ value. What happens if the pivot equals 0? How can we prevent that? **Answer:** PALU.

<div id='palu' />
### PALU decomposition

As you might've noted previously, LU has a problem when a _pivot_ has the value of $0$. To handle this problem, we add row permutations to the original LU algorithm. The procedure is as follows:

1. When visiting the row $j$, search for $\max(|a_{j,j}|,\ |a_{j+1,j}|,\ \ldots,\ |a_{N-1,j}|,\ |a_{N,j}|)$ (the maximum between the pivot and the entries below it).
2. If such maximum is $|a_{j,k}| \neq |a_{j,j}|$, permutate rows $i$ and $k$ making $a_{j,k}$ the new pivot.

To keep track of all the permutations performed, we use the permutation matrix $P$. It's inicially an identity matrix which permutes its rows in the same way the algorithm does on the resulting matrix. 

In [ ]:
#permutation between rows i and j on matrix A
def row_perm(A, i, j):
    tmp = np.copy(A[i])
    A[i] = A[j]
    A[j] = tmp

def palu_decomp(A, show=False):
    N,_ = A.shape
    P = np.identity(N)
    L = np.zeros((N,N))
    U = np.copy(A)
    if show:
        print('Initial matrices')
        print('P = '); print(np.array_str(P, precision=2, suppress_small=True))
        print('L = '); print(np.array_str(L, precision=2, suppress_small=True))
        print('U = '); print(np.array_str(U, precision=2, suppress_small=True))
        print('----------------------------------------')
    #iterating through columns
    for j in range(N-1):
        #determine the new pivot
        p_index = np.argmax(np.abs(U[j:,j]))
        if p_index != 0:
            row_perm(P, j, j+p_index)
            row_perm(U, j, j+p_index)
            row_perm(L, j, j+p_index)
            if show:
                print('A permutation has been made')
                print('P = '); print(np.array_str(P, precision=2, suppress_small=True))
                print('L = '); print(np.array_str(L, precision=2, suppress_small=True))
                print('U = '); print(np.array_str(U, precision=2, suppress_small=True))
                print('----------------------------------------')
        #iterating through rows
        for i in range(j+1,N):
            L[i,j] = U[i,j]/U[j,j]
            U[i] -= L[i,j]*U[j]
            if show:
                print('P = '); print(np.array_str(P, precision=2, suppress_small=True))
                print('L = '); print(np.array_str(L, precision=2, suppress_small=True))
                print('U = '); print(np.array_str(U, precision=2, suppress_small=True))
                print('----------------------------------------')
    np.fill_diagonal(L,1)
    return P,L,U

The procedure to solve the system $Ax=b$ remains almost the same. We have to add the efect of the permutation matrix $P$:

$$A x = b \rightarrow P A x = P b \rightarrow L U x = b' \ \ \text{ if we set } \ \  U x = c \rightarrow L c = b' \ \ \text{ (solve for c) } \ \rightarrow U x = c$$

In [ ]:
def solve_palu(A, b, show=False):
    P,L,U = palu_decomp(A, show)
    #A.x = b -> P.A.x = P.b = b'
    b = np.dot(P,b)
    # L.c = b' with c = U.x
    c = solve_triangular(L, b, upper=False)
    x = solve_triangular(U, c)
    return x

Let's test this new method against the LU and NumPy solvers

In [ ]:
palu_sol = solve_palu(A, b, show=True)

In [ ]:
np.linalg.norm(palu_sol - lu_sol)

In [ ]:
np.linalg.norm(palu_sol - np_sol)

Here are some questions about PALU:
1. How much computational complexity has been added to the original $2n^3/3$ of LU?
2. Clearly PALU is more robust than LU, but given a non sigular matrix $A$ will it always be possible to perform the PALU decomposition?

<div id='cholesky' />
### Cholesky

This is another direct method only applicable to _symmetric positive-definite_ matrices. In order to try this algorithm we have to create this kind of matrices. The next function generates random _symmetric positive-definite_ matrices. 

In [ ]:
"""
Randomly generates an nxn symmetric positive-
definite matrix A.
"""
def generate_spd_matrix(n):
    A = np.random.random((n,n))
    #constructing symmetry
    A += A.T
    #symmetric+diagonally dominant -> symmetric positive-definite
    deltas = 0.1*np.random.random(n)
    row_sum = A.sum(axis=1)-np.diag(A)
    np.fill_diagonal(A, row_sum+deltas)
    return A

Given a symmetric positive-definite matrix $A \in \mathbb{R}^{n \times n}$, the Cholesky decomposition is of the form $A =R^T R$, with $R$ being an upper triangular matrix. This method takes advantage of the properties of symmetric matrices, reaching approximately twice the efficiency of LU.

In [ ]:
def cholesky_decomp(A, show=False):
    N,_ = A.shape
    A = np.copy(A)
    R = np.zeros((N,N))
    if show:
        print('Initial matrix')
        print('A = '); print(np.array_str(A, precision=2, suppress_small=True))
        print('R = '); print(np.array_str(R, precision=2, suppress_small=True))
        print('----------------------------------------')
    for i in range(N):
        R[i,i] = np.sqrt(A[i,i])
        u = (1./R[i,i])*A[i,i+1:]
        R[i,i+1:] = u
        A[i+1:,i+1:] -= np.outer(u,u)
        if show:
            print('A = '); print(np.array_str(A, precision=2, suppress_small=True))
            print('R = '); print(np.array_str(R, precision=2, suppress_small=True))
            print('----------------------------------------')
    return R

The solve stage remains the same as LU:  

In [ ]:
def solve_cholesky(A, b, show=False):
    R = cholesky_decomp(A, show)
    #R^T.R.x = b -> R^T.c = b with R.x = c
    c = solve_triangular(R.T, b, upper=False)
    x = solve_triangular(R, c)
    return x

Now we test our implementation, comparing time execution with LU and PALU on two different linear systems

In [ ]:
A = generate_spd_matrix(3)
b = np.ones(3)

In [ ]:
solve_cholesky(A, b, show=True)

In [ ]:
A = generate_spd_matrix(100)
b = np.ones(100)

In [ ]:
%timeit solve_cholesky(A, b)
%timeit solve_lu(A, b)
%timeit solve_palu(A, b)

In [ ]:
A = generate_spd_matrix(1000)
b = np.ones(1000)

In [ ]:
%timeit solve_cholesky(A, b)
%timeit solve_lu(A, b)
%timeit solve_palu(A, b)

<div id='im' />
## Iterative Methods

In [ ]:
"""
Randomly generates an nxn strictly diagonally 
dominant matrix A.
"""
def generate_dd_matrix(n):
    A = np.random.random((n,n))
    deltas = 0.1*np.random.random(n)
    row_sum = A.sum(axis=1)-np.diag(A)
    np.fill_diagonal(A, row_sum+deltas)
    return A

"""
Computes relative error between each row on 
X matrix and y vector. 
"""
def error(X, y):
    D = X-y
    err = np.linalg.norm(D, axis=1, ord=np.inf)
    return err

As before we will create a linear system $A x = b$, with $A$ as a diagonally dominant matrix, which is a **sufficient** condition for the methods we will study in this section converge 

In [ ]:
A = np.array([[3, -1, 0, 0, 0, 0.5],[-1, 3, -1, 0, 0.5, 0],[0, -1, 3, -1, 0, 0],[0, 0, -1, 3, -1, 0],
              [0, 0.5, 0, -1, 3, -1],[0.5, 0, 0, 0, -1, 3]])
b = np.array([2.5, 1.5, 1., 1., 1.5, 2.5])
print ('A='); print (A)
print ('b='); print (b)

and find the solution $x$ through `np.linalg.solve` to use it as the reference solution-

In [ ]:
np_sol = np.linalg.solve(A,b)

### Jacobi

In [ ]:
"""
Iterative methods implementations returns an array X
with the the solutions at each iteration
"""
def jacobi(A, b, n_iter=50):
    n = A.shape[0]
    #array with solutions
    X = np.empty((n_iter, n))
    #initial guess
    X[0] = np.zeros(n)
    #submatrices
    D = np.diag(A)
    Dinv = D**-1
    R = A - np.diag(D) #R=(L+U)
    for i in range(1, n_iter):
        X[i] = Dinv*(b - np.dot(R, X[i-1]))
    return X

Now let's resolve the same linear system with Jacobi method!

In [ ]:
jac_sol = jacobi(A,b)

In [ ]:
jac_err = error(jac_sol, np_sol)
it = np.linspace(1, 50)

In [ ]:
plt.figure(figsize=(12,6))
plt.semilogy(it, jac_err, marker='o', linestyle='--', color='b')
plt.grid(True)
plt.xlabel('Iterations')
plt.ylabel('Error')
plt.title('Infinity norm error for Jacobi method')
plt.show()

### Gauss Seidel

In [ ]:
def gauss_seidel(A, b, n_iter=50):
    n = A.shape[0]
    #array with solutions
    X = np.empty((n_iter, n))
    #initial guess
    X[0] = np.zeros(n)
    #submatrices
    R = np.tril(A) #R=(L+D)
    U = A-R
    for i in range(1, n_iter):
        X[i] = solve_triangular(R, b-np.dot(U, X[i-1]), upper=False)
    return X

Now let's resolve the same linear system with Gauss-Seidel method!

In [ ]:
gauss_sol = gauss_seidel(A,b)

In [ ]:
gauss_err = error(gauss_sol, np_sol)

In [ ]:
plt.figure(figsize=(12,6))
plt.semilogy(it, gauss_err, marker='o', linestyle='--', color='r')
plt.grid(True)
plt.xlabel('Iterations')
plt.ylabel('Error')
plt.title('Infinity norm error for Gauss method')
plt.show()

Here are some questions about Gauss-Seidel:
- Can you explain what the differences between this and Jacobi method are?
- Why do we use `solve_triangular` instead of `np.linalg.solve` or something similar?

### SOR

In [ ]:
def sor(A, b, w=1.05, n_iter=50):
    n = A.shape[0]
    #array with solutions
    X = np.empty((n_iter, n))
    #initial guess
    X[0] = np.zeros(n)
    #submatrices
    R = np.tril(A) #R=(L+D)
    U = A-R
    for i in range(1, n_iter):
        X_i = solve_triangular(R, b-np.dot(U, X[i-1]), upper=False)
        X[i] = w*X_i + (1-w)*X[i-1]
    return X

Now let's resolve the same linear system with Jacobi method!

In [ ]:
sor_sol = sor(A, b, w=1.15)

In [ ]:
sor_err = error(sor_sol, np_sol)

In [ ]:
plt.figure(figsize=(12,6))
plt.semilogy(it, sor_err, marker='o', linestyle='--', color='g')
plt.grid(True)
plt.xlabel('Iterations')
plt.ylabel('Error')
plt.title('Infinity norm error for SOR method')
plt.show()

How can we choose a good value of $\omega$? Well there are  some methods you could search, but for now we will try a naive way, i.e, computing the solution for a range $\omega \in [1,1.3]$ as follows:

In [ ]:
n = 30 #width of subdivisions
sor_solutions = list()
for w in np.linspace(1., 1.3, n):
    sor_solutions.append(sor(A, b, w, n_iter=5)[-1])
np.asarray(sor_solutions)

#now compute error solutions with each w
sor_errors = error(sor_solutions, np_sol)
w = np.linspace(1., 1.3, n)

as you can see, we compute the SOR solution with 5 iterations for each $\omega$ on the given range. 

In [ ]:
plt.figure(figsize=(12,6))
plt.semilogy(w, sor_errors, marker='o', linestyle='--', color='g')
plt.grid(True)
plt.xlabel('w')
plt.ylabel('Error')
plt.title('Infinity norm error after 5 steps of SOR as a function of w')
plt.show()

Here are some questions about SOR:
- Why can averaging the current solution with the Gauss-Seidel solution improve convergence?
- Why do we use $\omega > 1$ and not $\omega < 1$?
- Could you describe a method to find the best value of $\omega$ (the one which optimizes convergence)?
- Would it be a better option to re-compute $\omega$ at each iteration?

<div id='ca' />
## Convergence Analysis

Let's see convergence plots all together

In [ ]:
plt.figure(figsize=(12,6))
plt.semilogy(it, jac_err, marker='o', linestyle='--', color='b', label='Jacobi')
plt.semilogy(it, gauss_err, marker='o', linestyle='--', color='r', label='Gauss-Seidel')
plt.semilogy(it, sor_err, marker='o', linestyle='--', color='g', label='SOR')
plt.grid(True)
plt.xlabel('Iterations')
plt.ylabel('Error')
plt.title('Infinity norm error for all methods')
plt.legend(loc=0)
plt.show()

<div id='ex' />
## Exercises

Now that you know how to solve systems of linear equations problem with these methods, let's try to answer a few questions!

$a)$ Find the values of $\alpha$ that make possible to do a LU descomposition of the following matrix:

$$ \begin{bmatrix}
   \alpha & 2  \\[0.3em]
   1 & \alpha  \end{bmatrix} $$

$b)$- Let $A$ be the following matrix:

$$ A = \begin{bmatrix}
        2 &  4 &  2  \\[0.3em]
       -1 &  1 &  2  \\[0.3em]
       -1 & -3 & -1  \end{bmatrix} $$ 
       
   - Find the PALU descomposition of the matrix $A$.
   
   - Solve the system of equations $Ax = \[1 , \frac{1}{2}, \frac{1}{3}\]^T$.

$c)$ Considering this matrix:

$$  \begin{bmatrix}
    1 & 1 & 0  \\[0.3em]
    1 & 5 & 2  \\[0.3em]
    0 & 2 & 3  \end{bmatrix} $$
       
   - Find the LU descomposition.
    
   - Find the Cholesky descomposition.
    
   - Compare the efficiency of both methods.

$d)$ Use Jacobi, Gauss Seidel, and SOR to solve the following system of equations (number of iterations = 2):

$$2x + y = 3$$
$$x + 2y = 2$$

   - Which is the best method to solve this problem (with better results)?

$e)$ Explain the pros and cons of using iterative methods instead of the direct ones.



## Extra: Hilbert Matrix

In [ ]:
from scipy.linalg import hilbert

In [ ]:
N=20
errors=np.zeros(N+1)
kappas=np.zeros(N+1)
my_range=np.arange(5,N+1)
for n in my_range:
    A=hilbert(n)
    x_exact=np.ones(n)
    b=np.dot(A,x_exact)
    x=np.linalg.solve(A,b)
    errors[n]=np.linalg.norm(x-x_exact,ord=np.inf)
    kappas[n]=np.linalg.cond(A,2)

In [ ]:
plt.figure(figsize=(12,6))
plt.semilogy(my_range, 10.**(-16+np.log10(kappas[my_range])), marker='o', linestyle='--', color='b',label='Estimated forward error')
plt.semilogy(my_range, errors[my_range], marker='o', linestyle='--', color='r',label='Forward error')
plt.grid(True)
plt.xlabel('n')
#plt.ylabel('$\kappa$ and errors')
plt.title('')
plt.legend(loc=0)
plt.show()

In [ ]:
n=20
A=hilbert(n)
x_exact=np.ones(n)
b=np.dot(A,x_exact)
x=np.linalg.solve(A,b)
kappa=np.linalg.cond(A,2)

In [ ]:
np.linalg.norm(b-np.dot(A,x))

In [ ]:
kappa

In [ ]:
np.linalg.norm(x-x_exact)

In [ ]:
x_exact[0]

In [ ]:
x[-1]

In [ ]:
plt.plot(x,'.')
plt.show()

<div id='acknowledgements' />
# Acknowledgements
* _Material created by professor Claudio Torres_ (`ctorres@inf.utfsm.cl`) _and assistants: Laura Bermeo, Alvaro Salinas, Axel Simonsen and Martín Villanueva. DI UTFSM. April 2016._

In [ ]:
lu_decomp(np.array([[1e-20, 1],[1,2]]), show=True)